In [1]:
import math
from itertools import combinations, product

file_location = './data/tsp_100_3'

In [2]:
import sys
# helper function to read the input data
def read_input(input_data):
    lines = input_data.split('\n')

    nodeCount = int(lines[0])

    points = []
    for i in range(1, nodeCount+1):
        line = lines[i]
        parts = line.split()
        points.append((float(parts[0]), float(parts[1])))
    
    return points, nodeCount

input_data = None
with open(file_location, 'r') as input_data_file:
    input_data = input_data_file.read()

points, nodeCount = read_input(input_data)
print(nodeCount)

100


In [3]:
def distance(point1, point2):
    dims = len(point1)
    return math.sqrt(sum((point1[k] - point2[k])**2 for k in range(dims)))

In [4]:
# helper function to determine the distance between two points
dist = {(i, j):
        math.sqrt(sum((points[i][k]-points[j][k])**2 for k in range(2)))
        for i in range(nodeCount) for j in range(i)}

In [5]:
def subtourelim(model, where):
    if where == GRB.Callback.MIPSOL:
        # make a list of edges selected in the solution
        vals = model.cbGetSolution(model._vars)
        selected = gp.tuplelist((i, j) for i, j in model._vars.keys()
                                if vals[i, j] > 0.5)
        # find the shortest cycle in the selected edge list
        tour = subtour(selected)
        if len(tour) < nodeCount:
            # add subtour elimination constr. for every pair of cities in tour
            model.cbLazy(gp.quicksum(model._vars[i, j]
                                     for i, j in combinations(tour, 2))
                         <= len(tour)-1)

In [6]:
def subtour(edges):
    unvisited = list(range(nodeCount))
    cycle = range(nodeCount+1)  # initial length has 1 more city
    while unvisited:  # true if list is non-empty
        thiscycle = []
        neighbors = unvisited
        while neighbors:
            current = neighbors[0]
            thiscycle.append(current)
            unvisited.remove(current)
            neighbors = [j for i, j in edges.select(current, '*')
                         if j in unvisited]
        if len(cycle) > len(thiscycle):
            cycle = thiscycle
    return cycle

In [7]:
import gurobipy as gp
from gurobipy import GRB

def solve_it(input_data):
    # define model
    m = gp.Model()
    # add variables
    vars = m.addVars(dist.keys(), obj=dist, vtype=GRB.BINARY, name='e')
    # add constraints : adjacency
    for i, j in vars.keys():
        vars[j, i] = vars[i, j]  # edge in opposite direction
    # add constraints: every point has two adjacent points
    m.addConstrs(vars.sum(i, '*') == 2 for i in range(nodeCount))
    m._vars = vars
    m.Params.lazyConstraints = 1
    m.optimize(subtourelim)
    
    # retrieve solution
    vals = m.getAttr('x', vars)
    selected = gp.tuplelist((i, j) for i, j in vals.keys() if vals[i, j] > 0.5)

    solution = subtour(selected)
    
    print("Solution: ", solution)
    
    # calculate the length of the tour
    obj = m.objVal

    # prepare the solution in the specified output format
    output_data = '%.2f' % obj + ' ' + str(0) + '\n'
    output_data += ' '.join(map(str, solution))

    return output_data

In [8]:
print(solve_it(input_data))

Using license file /Users/severinpfister/gurobi.lic
Academic license - for non-commercial use only
Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 100 rows, 4950 columns and 9900 nonzeros
Model fingerprint: 0xd8cc650c
Variable types: 0 continuous, 4950 integer (4950 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 4e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Presolve time: 0.01s
Presolved: 100 rows, 4950 columns, 9900 nonzeros
Variable types: 0 continuous, 4950 integer (4950 binary)

Root relaxation: objective 1.970326e+04, 152 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 19703.2605    0   12          - 19703.2605      -     -    0s
     0     0 19880.2210 